## Загрузка необходимых библиотек

In [1]:
%pip install deeppavlov
%pip install transformers
%pip install torch
%pip install tqdm
%pip install fastparquet
# %pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 748.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 1.8 MB/s eta 0:00:0000:010:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# Загрузка данных и их обработка

In [2]:
import pandas as pd
video_title = pd.read_parquet(
    'train_data/train_data/videos.parquet',
    engine='fastparquet',
    columns=['video_id', 'video_title']
)

print("video_title загружены")

video_title загружены


In [3]:
from collections import defaultdict

automarkup = pd.read_parquet(
    'train_data/train_data/automarkup.parquet',
    engine='fastparquet',
    columns=['video_id', 'query']
)
automarkup = automarkup[:1000000]
video_id = []
dict_video = defaultdict(str)


dict_array = defaultdict(list)
for i, j in zip(automarkup['video_id'], automarkup['query']):
    dict_array[j] = i

video_title = video_title.loc[video_title['video_id'].isin(automarkup['video_id'])]

for i, j in zip(video_title['video_id'], video_title['video_title']):
    dict_video[i] = j

text_query = []
text_title = []

for key, value in dict_array.items():
    text_query.append({"relevance":1, "query":key.lower()})
    text_title.append(dict_video[value])

print("text_title создана")

text_title создана


In [4]:
# Словарь где ключ это название видео
video_dict = dict()

for i,j in zip(video_title['video_id'], video_title['video_title']):
    video_dict[j]=i

print("video_dict создана")

video_dict создана


# Обучение модели

In [5]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm

class CustomDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {'text': self.texts[idx], 'label': self.labels[idx]}

# Объединение данных для всех запросов
all_train_texts = text_title# + train_texts_python + train_texts_web
all_train_labels = text_query# + train_labels_python + train_labels_web

# Создание DataLoader
train_dataset = CustomDataset(all_train_texts, all_train_labels)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

# Загрузка предварительно обученного токенизатора и модели BERT для русского языка
model_name = 'cointegrated/rubert-tiny2'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)

# Настройка оптимизатора и функции потерь
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.BCEWithLogitsLoss()

# Обучение
num_epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Задайте accumulation_steps до цикла обучения
accumulation_steps = 1  # Вы можете изменить это значение на необходимое

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    # добавлено оборачивание в tqdm
    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}")):
        inputs = tokenizer(batch['text'], return_tensors='pt', padding=True, truncation=True).to(device)
        # print(batch['label'])
        relevance_labels = [item for item in batch['label']['relevance']]

        labels = torch.tensor(relevance_labels, dtype=torch.float).unsqueeze(1).to(device)

        outputs = model(**inputs).logits
        loss = criterion(outputs, labels)
        loss.backward()

        # Добавляем gradient accumulation
        if (batch_idx + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        total_loss += loss.item()
        # print(relevance_labels)

    # После завершения gradient accumulation, выполните еще одно обновление весов
    if len(train_loader) % accumulation_steps != 0:
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss}")



# Сохранение дообученной модели
model.save_pretrained("tune_model_d_fast_2")
tokenizer.save_pretrained("tune_model_d_fast_2")

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

...

Epoch 2/3: 100%|██████████| 395/395 [00:34<00:00, 11.61it/s]


Epoch 2/3, Loss: 0.01715910083178101


Epoch 3/3: 100%|██████████| 395/395 [00:33<00:00, 11.79it/s]

Epoch 3/3, Loss: 0.007859150457042682


('tune_model_d_fast_2/tokenizer_config.json',
 'tune_model_d_fast_2/special_tokens_map.json',
 'tune_model_d_fast_2/vocab.txt',
 'tune_model_d_fast_2/added_tokens.json')

# Загрузка тестового датасета

In [6]:
import pandas as pd

csv_file_path = 'submission/test_dataset_submission_queries.csv'
df = pd.read_csv(csv_file_path)
list_of_rows = df.values.tolist()


# Подборка релевантных видео и запись итогового результата в CSV формат

In [8]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import time
import csv


#CLS Pooling - Take output from first token
def cls_pooling(model_output):
    return model_output.last_hidden_state[:,0]

#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = cls_pooling(model_output)

    return embeddings

def csv_writer(output_file_path):

    with open(output_file_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

        csv_writer.writerow(['query', 'video_id'])

    print(f"CSV файл успешно создан: {output_file_path}")


def csv_writer_submission(output_file_path, list_to_csv):

    with open(output_file_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['query', 'video_id'])

        for row in list_to_csv:
            csv_writer.writerow(row)

    print(f"CSV файл успешно создан: {output_file_path}")

def embedding_text(text_title,tokenizer,model):
    # video_data = {"video_id":[i for i in range(1, len(set(text_title)))],"text":list(set(text_title))}
    data = list(text_title.keys())
    text_embeddings = encode(data)

    return text_embeddings, data
def main(user_query, text_embeddings, tokenizer,model, data, video_dict):
    start_time = time.time()
    user_query_original = user_query
    user_query = user_query.lower()
    query_emb = encode(query)

    # Вычисление косинусной схожести
    cosine_similarities = cosine_similarity(query_emb, text_embeddings)

    # Вывод наиболее релевантных текстовых данных
    print(f"Ваш запрос: {user_query}")
    print("Наиболее релевантные тексты:")

    # Создайте список кортежей из video_id и cosine_similarities
    results = [(video_data, similarity) for video_data, similarity in zip(data, cosine_similarities.flatten())]

    # Упорядочьте результаты по убыванию косинусной схожести
    sorted_results = sorted(results, key=lambda x: x[1], reverse=True)

    # Выведите упорядоченные результаты
    list_to_csv = []

    for video_title, similarity in sorted_results[:5]:
        print(f"ID видео: {video_dict[video_title]}, Косинусная схожесть: {similarity}")
        list_to_csv.append((user_query_original, video_dict[video_title]))

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Время выполнения: {elapsed_time} секунд")

    return list_to_csv


if __name__ == "__main__":
    # Загрузка предварительно обученного токенизатора и модели BERT
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    model = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

    output_file_path = 'submission/sample_sabmission.csv'
    exit_list_to_csv = []
    text_embeddings, data = embedding_text(video_dict, tokenizer,model )
    for query in list_of_rows:
        _list = main(query[0],text_embeddings,tokenizer,model, data, video_dict)
        exit_list_to_csv+=_list
    #csv_writer(output_file_path=output_file_path)
    csv_writer_submission(
        output_file_path,
        exit_list_to_csv)


Ваш запрос: битва сильнейших экстрасенсов 2023 смотреть | экстрасенсы битва сильнейших 35, 36 серия
Наиболее релевантные тексты:
ID видео: video_19233886, Косинусная схожесть: 0.9220086336135864
ID видео: video_18639714, Косинусная схожесть: 0.9216121435165405
ID видео: video_21665327, Косинусная схожесть: 0.908867597579956
ID видео: video_6946688, Косинусная схожесть: 0.9079782366752625
ID видео: video_31472773, Косинусная схожесть: 0.9067044854164124
Время выполнения: 0.02966928482055664 секунд
Ваш запрос: битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023 смотреть
Наиболее релевантные тексты:
ID видео: video_19233886, Косинусная схожесть: 0.919266402721405
ID видео: video_28129270, Косинусная схожесть: 0.913249135017395
ID видео: video_19577401, Косинусная схожесть: 0.9101969003677368
ID видео: video_6946688, Косинусная схожесть: 0.9093248248100281
ID видео: video_34141741, Косинусная схожесть: 0.9070925712585449
Время выполнения: 0.1647629737854004 секунд
Ваш за